In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import optim, nn
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('data/beijing_dataset.csv')

In [3]:
df

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [4]:
df.describe()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir
count,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,41757.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000
mean,21912.500000,2012.000000,6.523549,15.727820,11.500000,98.613215,1.817246,12.448521,1016.447654,23.889140,0.052734,0.194916
std,12651.043435,1.413842,3.448572,8.799425,6.922266,92.050387,14.433440,12.198613,10.268698,50.010635,0.760375,1.415867
min,1.000000,2010.000000,1.000000,1.000000,0.000000,0.000000,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000
25%,10956.750000,2011.000000,4.000000,8.000000,5.750000,29.000000,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000
50%,21912.500000,2012.000000,7.000000,16.000000,11.500000,72.000000,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000
75%,32868.250000,2013.000000,10.000000,23.000000,17.250000,137.000000,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000
max,43824.000000,2014.000000,12.000000,31.000000,23.000000,994.000000,28.000000,42.000000,1046.000000,585.600000,27.000000,36.000000


### Fix the datetime from 4 columns to 1.

In [9]:
df['datetime'] = pd.to_datetime(df[["year", "month", "day", "hour"]])

In [13]:
df.set_index('datetime', inplace=True)
df.drop(['No', 'year', 'month', 'day', 'hour'], axis=1, inplace=True)

In [14]:
df

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
datetime,,,,,,,,
2010-01-01 00:00:00,NaN,-21,-11.0,1021.0,NW,1.79,0,0
2010-01-01 01:00:00,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2010-01-01 02:00:00,NaN,-21,-11.0,1019.0,NW,6.71,0,0
2010-01-01 03:00:00,NaN,-21,-14.0,1019.0,NW,9.84,0,0
2010-01-01 04:00:00,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,NW,231.97,0,0
2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,NW,237.78,0,0
2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,NW,242.70,0,0


### Fixing out the Nan Values

In [16]:
df.isnull().sum()

pm2.5    2067
DEWP        0
TEMP        0
PRES        0
cbwd        0
Iws         0
Is          0
Ir          0
dtype: int64

In [20]:
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

/var/folders/pl/cdtxfpj90yv22wr7bnk7kyxh0000gn/T/ipykernel_76483/3490755982.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
/var/folders/pl/cdtxfpj90yv22wr7bnk7kyxh0000gn/T/ipykernel_76483/3490755982.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')


In [21]:
df.isnull().sum()

pm2.5    0
DEWP     0
TEMP     0
PRES     0
cbwd     0
Iws      0
Is       0
Ir       0
dtype: int64

In [22]:
df

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
datetime,,,,,,,,
2010-01-01 00:00:00,129.0,-21,-11.0,1021.0,NW,1.79,0,0
2010-01-01 01:00:00,129.0,-21,-12.0,1020.0,NW,4.92,0,0
2010-01-01 02:00:00,129.0,-21,-11.0,1019.0,NW,6.71,0,0
2010-01-01 03:00:00,129.0,-21,-14.0,1019.0,NW,9.84,0,0
2010-01-01 04:00:00,129.0,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,NW,231.97,0,0
2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,NW,237.78,0,0
2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,NW,242.70,0,0


In [25]:
df = pd.get_dummies(df, columns=['cbwd'])

In [26]:
df

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,cbwd_NE,cbwd_NW,cbwd_SE,cbwd_cv
datetime,,,,,,,,,,,
2010-01-01 00:00:00,129.0,-21,-11.0,1021.0,1.79,0,0,False,True,False,False
2010-01-01 01:00:00,129.0,-21,-12.0,1020.0,4.92,0,0,False,True,False,False
2010-01-01 02:00:00,129.0,-21,-11.0,1019.0,6.71,0,0,False,True,False,False
2010-01-01 03:00:00,129.0,-21,-14.0,1019.0,9.84,0,0,False,True,False,False
2010-01-01 04:00:00,129.0,-20,-12.0,1018.0,12.97,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,False,True,False,False
2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,False,True,False,False
2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,False,True,False,False


In [33]:
features = df.drop(columns = ["pm2.5"])
target = df["pm2.5"]

In [34]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Combine scaled features with target for sequence creation
data = np.hstack((features_scaled, target.values.reshape(-1, 1)))

In [38]:
features_scaled[0]

array([0.27941176, 0.13114754, 0.54545455, 0.00229001, 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ])

In [36]:
data[0]

array([2.79411765e-01, 1.31147541e-01, 5.45454545e-01, 2.29001111e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.29000000e+02])

In [62]:
X = []
y = []
seq_length = 30
for i in range(len(df) - seq_length):
    X.append(data[i:i+seq_length, :-1])
    y.append(data[i+seq_length, -1])

X = np.array(X)
y = np.array(y)

In [65]:
X_tensor = torch.tensor(X, dtype=torch.float32)# .unsqueeze(-1)  # (batch, seq_len, 1)
Y_tensor = torch.tensor(y, dtype=torch.float32)#.unsqueeze(-1)   # (batch, 1)

In [66]:
X_tensor.shape

torch.Size([43794, 30, 10])

In [51]:
class ManualLSTM():
    def __init__(self, input_size, hidden_size):
        self.learning_rate = 0.01
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.W_xf = torch.randn( input_size, hidden_size, requires_grad=True, device = device)
        self.W_xi = torch.randn( input_size, hidden_size, requires_grad=True, device = device) 
        self.W_xg = torch.randn( input_size, hidden_size, requires_grad=True, device = device)
        self.W_xo = torch.randn( input_size, hidden_size, requires_grad=True, device = device)
        self.W_hf = torch.randn(hidden_size, hidden_size, requires_grad=True, device = device)
        self.W_hi = torch.randn(hidden_size, hidden_size, requires_grad=True, device = device)
        self.W_hg = torch.randn(hidden_size, hidden_size, requires_grad=True, device = device)
        self.W_ho = torch.randn(hidden_size, hidden_size, requires_grad=True, device = device)

        self.b_f = torch.randn(hidden_size, requires_grad=True, device = device)
        self.b_i = torch.randn(hidden_size, requires_grad=True, device = device)
        self.b_g = torch.randn(hidden_size, requires_grad=True, device = device)
        self.b_o = torch.randn(hidden_size, requires_grad=True, device = device)

        output_size = 1
        self.W_out = torch.randn(hidden_size, output_size, requires_grad=True, device = device)
        self.b_out = torch.randn(output_size, requires_grad=True, device = device)
    
    def forward(self, x_t_seq):
        
        h_t = torch.zeros(1, self.hidden_size, device= device)
        c_t = torch.zeros(1, self.hidden_size, device= device)
        
    
        for t in range(len(x_t_seq)):                 # Loop over time steps
            x_t = x_t_seq[t].unsqueeze(0)         # Shape: (1, input_size)
        
            f_t = torch.sigmoid(x_t @ self.W_xf + h_t @ self.W_hf + self.b_f)
            i_t = torch.sigmoid(x_t @ self.W_xi + h_t @ self.W_hi + self.b_i)
            g_t = torch.tanh(   x_t @ self.W_xg + h_t @ self.W_hg + self.b_g)
            o_t = torch.sigmoid(x_t @ self.W_xo + h_t @ self.W_ho + self.b_o)

            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
        return h_t @ self.W_out + self.b_out
    
    def step(self):
        with torch.no_grad():
            for params in [self.W_xf, self.W_xi, self.W_xg, self.W_xo,
                            self.W_hf, self.W_hi, self.W_hg, self.W_ho,
                            self.b_f, self.b_i, self.b_g, self.b_o,
                            self.W_out, self.b_out]:
                
                params -= self.learning_rate * params.grad
                params.grad.zero_()
                
    def save(self, filename):
        torch.save(self.__dict__, filename)
    
    def load(self, filename):
        state = torch.load(filename, map_location = device)
        self.__dict__.update(state)
    
    

In [67]:
batch_size, seq_size, input_size = X_tensor.shape

In [68]:
X_tensor.shape

torch.Size([43794, 30, 10])

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [74]:
model = ManualLSTM(input_size = input_size, hidden_size = 5)

In [ ]:
loss_fn = nn.MSELoss()

learning_rate = 0.01
epochs = 30

for n in range(epochs):
    total_loss = 0.0
    for i in range(batch_size):                    # Loop over samples
        x_t_seq = X_tensor[i]                     # Shape: (seq_len, input_size)
        
        
        # print(x_t_seq.shape)
        y_pred = model.forward(x_t_seq)
        y_true = Y_tensor[i].view(1,1)
    
        loss = loss_fn(y_pred, y_true)
    
        total_loss += loss
    
        loss.backward()
        
        model.step()
        
        if i%500 == 0:
            print(i)
            
    
    
    print(f"Epochs {n}: Total Loss = {total_loss:.6f}")

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
